# Données WFP - Admin 1

In [2]:
import sys
import os

# Détection automatique de la racine du projet
# On remonte d'un niveau depuis le dossier 'notebooks'
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ajout au path pour pouvoir faire "from src..."
if project_root not in sys.path:
    sys.path.append(project_root)

# Définition des chemins absolus
DATA_RAW = os.path.join(project_root, "data", "raw")
DATA_PROCESSED = os.path.join(project_root, "data", "processed")
GEO_PATH = os.path.join(project_root, "geo")

print(f"Système configuré. Racine : {project_root}")

Système configuré. Racine : c:\Users\valen\OneDrive\Documents\GitHub\Projet-de-Statapp---WFP-


In [3]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURATION DES CHEMINS
# ==============================================================================
# Ton fichier météo (généré à l'étape précédente)
fldas_path = "../data/raw/FLDAS_Nigeria_Admin2_Aggregated.csv" 
# Le fichier WFP 
wfp_path = "../data/raw/WFP_food_security_data_nigeria.csv"
# Fichier de sortie
output_path = "../data/processed/NIGERIA_WFP_ADMIN1.csv"


In [7]:
# ==============================================================================
# 2. TRAITEMENT CLIMAT (ADMIN 2 -> ADMIN 1)
# ==============================================================================
print("1. Chargement et agrégation des données climatiques...")
df_clim = pd.read_csv(fldas_path)
df_clim['time'] = pd.to_datetime(df_clim['time'])

#On regroupe par DATE et par ÉTAT (NAME_1)
# On fait la moyenne de toutes les LGA (NAME_2) qui sont dans cet état
df_clim_admin1 = df_clim.groupby(['time', 'NAME_1']).mean(numeric_only=True).reset_index()


# 2. CORRECTION DES NOMS 
df_clim_admin1['NAME_1'] = df_clim_admin1['NAME_1'].replace({
    'FederalCapitalTerritory': 'Abuja',  # Le plus important
    'AkwaIbom': 'Akwa Ibom',             # Manque l'espace
    'CrossRiver': 'Cross River',         # Manque l'espace
    'Nasarawa': 'Nassarawa',             # Orthographe avec 2 's' souvent utilisée dans WFP
    'Nassarawa': 'Nassarawa'             # Sécurité si déjà avec 2 s
})

print(f"   -> Données climatiques agrégées au niveau État : {df_clim_admin1.shape}")


1. Chargement et agrégation des données climatiques...
   -> Données climatiques agrégées au niveau État : (2220, 30)


In [8]:

# ==============================================================================
# 3. TRAITEMENT WFP (CIBLE)
# ==============================================================================
print("2. Préparation des données WFP...")
df_wfp = pd.read_csv(wfp_path)

# Création d'une vraie colonne Date (1er du mois)
# On prend l'année du 'quarter' (ex: "2023Q1" -> 2023) et le mois
df_wfp['year'] = df_wfp['quarter'].str[:4].astype(int)
df_wfp['time'] = pd.to_datetime(df_wfp[['year', 'month']].assign(day=1))

# On renomme pour avoir la même clé de jointure
df_wfp = df_wfp.rename(columns={'ADM1_NAME': 'NAME_1'})

# On garde les colonnes utiles
cols_wfp = ['time', 'NAME_1', 'inadequate']
df_wfp_clean = df_wfp[cols_wfp]

print(f"   -> Données WFP prêtes : {df_wfp_clean.shape}")


2. Préparation des données WFP...
   -> Données WFP prêtes : (1010, 3)


In [9]:

# ==============================================================================
# 4. FUSION (MERGE)
# ==============================================================================
print("3. Fusion des deux fichiers...")

# Inner join : on garde uniquement les lignes où on a LA MÉTÉO ET LA CIBLE
df_final = pd.merge(df_wfp_clean, df_clim_admin1, on=['time', 'NAME_1'], how='inner')

# Sauvegarde
df_final.to_csv(output_path, index=False)

print("-" * 30)
print(f"✅ SUCCÈS ! Fichier fusionné sauvegardé : {output_path}")
print(f"Dimensions finales : {df_final.shape}")
print(f"Période couverte : {df_final['time'].min().date()} au {df_final['time'].max().date()}")
print("-" * 30)

# Aperçu
display(df_final.head())

3. Fusion des deux fichiers...
------------------------------
✅ SUCCÈS ! Fichier fusionné sauvegardé : ../data/processed/NIGERIA_WFP_ADMIN1.csv
Dimensions finales : (644, 31)
Période couverte : 2023-01-01 au 2024-12-01
------------------------------


,time,NAME_1,inadequate,Evap_tavg,LWdown_f_tavg,Lwnet_tavg,Psurf_f_tavg,Qair_f_tavg,Qg_tavg,Qh_tavg,...,Tair_f_tavg,Wind_f_tavg,SoilMoi00_10cm_tavg,SoilMoi10_40cm_tavg,SoilMoi40_100cm_tavg,SoilMoi100_200cm_tavg,SoilTemp00_10cm_tavg,SoilTemp10_40cm_tavg,SoilTemp40_100cm_tavg,SoilTemp100_200cm_tavg
0,2023-01-01,Abia,0.296774,0.000016,385.566324,-71.064699,99941.271400,0.013670,1.535858,52.585289,...,299.067207,3.369548,0.280146,0.364495,0.374807,0.389133,300.138219,299.605325,299.121555,298.700373
1,2023-01-01,Abuja,0.263514,0.000003,342.886723,-111.296370,97471.833500,0.007886,2.633969,91.256501,...,298.474872,4.564216,0.170063,0.352046,0.357529,0.371979,300.143308,299.459487,298.885082,298.513978
2,2023-01-01,Adamawa,0.340483,0.000006,327.309607,-120.776627,96184.946619,0.005087,2.256282,75.890667,...,298.876978,5.478152,0.151770,0.307479,0.318816,0.360368,299.263407,298.643008,298.328045,298.392429
3,2023-01-01,Akwa Ibom,0.342105,0.000026,396.777694,-60.200416,100550.291276,0.015601,1.257305,25.840834,...,299.744141,3.333093,0.324158,0.356986,0.371353,0.386222,300.036104,299.642669,299.196288,298.773942
4,2023-01-01,Bauchi,0.532895,0.000007,314.941737,-117.574605,95708.880950,0.004648,1.426913,59.048999,...,296.026673,6.130964,0.149414,0.321856,0.330437,0.358364,296.697754,296.368762,296.427992,297.035517
